In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [4]:
import os
import pandas as pd

journal = input('scientometrics / jasist')
entity = input('co-author / co-occurrence')

path = os.getcwd()
evaluation_path = path+"\\output prediction\\{}\\{}\\evaluation\\".format(journal, entity)
result_path = path+"\\output prediction\\{}\\{}\\result\\".format(journal, entity)
betweenness_path = path+"\\output prediction\\{}\\{}\\".format(journal, entity)+"edge_betweenness.txt"

scientometrics / jasistscientometrics
co-author / co-occurrenceco-occurrence


In [3]:
netfiles_path = path+"\\data_transformation_output\\{}\\netfiles\\{}\\".format(journal, entity)
netfiles = []
for f in os.listdir(netfiles_path):
    if f.endswith('.net'):
        netfiles.append(netfiles_path+f)

netfiles

['C:\\Users\\Liser\\Desktop\\linchengwei_link_prediction\\data_transformation_output\\scientometrics\\netfiles\\co-occurrence\\2010-15.net',
 'C:\\Users\\Liser\\Desktop\\linchengwei_link_prediction\\data_transformation_output\\scientometrics\\netfiles\\co-occurrence\\2016-16.net',
 'C:\\Users\\Liser\\Desktop\\linchengwei_link_prediction\\data_transformation_output\\scientometrics\\netfiles\\co-occurrence\\2016-17.net',
 'C:\\Users\\Liser\\Desktop\\linchengwei_link_prediction\\data_transformation_output\\scientometrics\\netfiles\\co-occurrence\\2016-18.net',
 'C:\\Users\\Liser\\Desktop\\linchengwei_link_prediction\\data_transformation_output\\scientometrics\\netfiles\\co-occurrence\\2016-19.net']

# read result

In [2]:
result = dict()

for f in os.listdir(result_path):
    df_iter = pd.read_csv(result_path+f, sep='\t', header=None)
    df_iter.columns=['keyword1', 'keyword2', f.split('.')[0]]
    result[f.split('.')[0]] = df_iter
    print(f)

NameError: name 'os' is not defined

# read evaluation

In [5]:
eva = dict()
for f in os.listdir(evaluation_path):
    df_iter = pd.read_csv(evaluation_path+f, sep=' ', header=None)
    df_iter.columns = ['tp', 'fp', 'fn', 'tn']
    eva[f.split('.')[0]] = df_iter
    print(f)

AdamicAdar_t1.txt
jaccard_t1.txt
katz_beta001_t1.txt
katz_beta01_t1.txt


# read Betweenness

In [6]:
df_betweenness = pd.read_csv(betweenness_path, sep='\t', header=None)
df_betweenness.columns = ['keyword1', 'keyword2', 'betweenness']
df_betweenness = df_betweenness.set_index(['keyword1', 'keyword2'])

### concat 4 results

In [8]:
df = pd.DataFrame()
for i, df_iter in enumerate(list(result.values())):
    df_iter = df_iter.set_index(['keyword1', 'keyword2'])
    df = pd.concat([df, df_iter], axis=1, sort=False)
#     print(i)

### concat 4 results with betweenness

In [9]:
df = pd.concat([df, df_betweenness], axis=1, sort=False)

In [17]:
df = df.reset_index()

### save

In [18]:
df.to_pickle('output_scoresheet\\scoresheet.pkl')

In [19]:
for k in eva.keys():
    print(k)

AdamicAdar_t1
jaccard_t1
katz_beta001_t1
katz_beta01_t1


## nodes in testing phase
1. find all possible links

In [7]:
import linkpred
G = linkpred.read_network(netfiles[0]) # t1
n = len(list(G.nodes))
print("{} nodes in {}".format(n, netfiles[0]))

all_possible_links = (n*(n-1))/2
print("all possible links: {}".format(all_possible_links))

Network contains multiple edges. These will be ignored.


4351 nodes in C:\Users\Liser\Desktop\linchengwei_link_prediction\data_transformation_output\scientometrics\netfiles\co-occurrence\2010-15.net
all possible links: 9463425.0


## computing confusion and new evaluation with correct tn

In [8]:
import numpy as np

def compute_confusion(evaluation, num_linked, num_of_nolink):
    
    def add_0_for_cal(X):
        
        C = pd.DataFrame()
        C = C.append([[0, 0, num_linked, num_of_nolink]])
        C.columns = X.columns
        X = C.append(X)
        
        return X

    X = add_0_for_cal(evaluation)[:-1]
    Y = add_0_for_cal(evaluation)[1:]
    X = X.reset_index()
    X = X.drop(columns=['index'])
    res = Y-X

    return res

df_confusions = dict()

for k in eva.keys():
    print('-'*10, k)
    
    # evaluation = eva[k]
    num_linked = int(eva[k]['tp'][0]+eva[k]['fn'][0])
    
    # calculate tn
    num_of_nolink = all_possible_links-num_linked
    print("num of linked:  {}".format(num_linked))
    print("num of no link: {}".format(num_of_nolink))
    
    df_confusion = compute_confusion(eva[k], num_linked, num_of_nolink)
    df_confusions[k] = df_confusion
    
    # num_of_nolink = np.array([num_of_nolink]*len(eva[k]['fp']))
    
    eva[k]['tn'] = np.array([num_of_nolink]*len(eva[k]['fp']))-eva[k]['fp']

---------- AdamicAdar_t1
num of linked:  3772
num of no link: 9459653.0
---------- jaccard_t1
num of linked:  3772
num of no link: 9459653.0
---------- katz_beta001_t1
num of linked:  3772
num of no link: 9459653.0
---------- katz_beta01_t1
num of linked:  3772
num of no link: 9459653.0


In [9]:
eva['AdamicAdar_t1']

,tp,fp,fn,tn
0,1.0,0.0,3771.0,9459653.0
1,1.0,1.0,3771.0,9459652.0
2,1.0,2.0,3771.0,9459651.0
3,1.0,3.0,3771.0,9459650.0
4,1.0,4.0,3771.0,9459649.0
...,...,...,...,...
512426,432.0,511995.0,3340.0,8947658.0
512427,432.0,511996.0,3340.0,8947657.0
512428,432.0,511997.0,3340.0,8947656.0
512429,432.0,511998.0,3340.0,8947655.0


## calculate sensitivity, specificity and YoundenIndex

In [21]:
x['sensitivity']+x['specificity']

0          1.000000
1          1.000265
2          1.000265
3          1.000265
4          1.000265
             ...   
7855068    0.390815
7855069    0.390815
7855070    0.390815
7855071    0.390815
7855072    0.390815
Length: 7855073, dtype: float64

In [22]:
for k in eva.keys():
    print('-'*10, k)
    x = eva[k]
    x['sensitivity'] = x['tp'] / (x['tp'] + x['fn'])
    x['specificity'] = x['tn'] / (x['tn'] + x['fp'])

    x['YoundenIndex'] = x['sensitivity']+x['specificity']-1
    
    Younden_idxmax = x['YoundenIndex'].idxmax()
    print(x.loc[Younden_idxmax], '\n')

---------- AdamicAdar_t1
tp              3.680000e+02
fp              2.837750e+05
fn              3.404000e+03
tn              9.175878e+06
sensitivity     9.756098e-02
specificity     9.700015e-01
YoundenIndex    6.756252e-02
Name: 284142, dtype: float64 

---------- jaccard_t1
tp              4.320000e+02
fp              5.119930e+05
fn              3.340000e+03
tn              8.947660e+06
sensitivity     1.145281e-01
specificity     9.458761e-01
YoundenIndex    6.040423e-02
Name: 512424, dtype: float64 

---------- katz_beta001_t1
tp              3.900000e+02
fp              2.064410e+05
fn              3.382000e+03
tn              9.253212e+06
sensitivity     1.033934e-01
specificity     9.781767e-01
YoundenIndex    8.157011e-02
Name: 206830, dtype: float64 

---------- katz_beta01_t1
tp              4.090000e+02
fp              2.445200e+05
fn              3.363000e+03
tn              9.215133e+06
sensitivity     1.084305e-01
specificity     9.741513e-01
YoundenIndex    8.258181

In [23]:
predict_results = dict()
for k in result.keys():
    predict_results[k] = result[k].copy()
    predict_results[k]['predict'] = np.array([1]*len(result[k]))
    predict_results[k]['actual'] = df_confusions[k]['tp']

In [24]:
all_results = dict()

for k in result.keys():
    all_results[k] = predict_results[k].join(eva[k])

### concat with betweenness

In [28]:
df_betweenness = df_betweenness.reset_index()
df_betweenness

,keyword1,keyword2,betweenness
0,All-in-one indicator,Altmetric score,0.000001
1,All-in-one indicator,Altmetrics,0.001344
2,1 Nano enabled drug delivery,Bibliography analysis,0.000001
3,1 Nano enabled drug delivery,Graphene,0.000007
4,1 Nano enabled drug delivery,Specialization score,0.000001
...,...,...,...
3767,fa(k)-index,fp(k)-index,0.000001
3768,fa(k)-index,fas(k)-index,0.000001
3769,fas(k)-index,fp(k)-index,0.000001
3770,g-index,h-index,0.000361


In [97]:
z = pd.merge(left, right, how='outer', on=['keyword1', 'keyword2'])

In [100]:
z[z['betweenness'].astype(str)!='nan']

,keyword1,keyword2,katz_beta001_t1,predict,actual,tp,fp,fn,tn,sensitivity,specificity,YoundenIndex,betweenness
7855073,All-in-one indicator,Altmetric score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001
7855074,All-in-one indicator,Altmetrics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001344
7855075,1 Nano enabled drug delivery,Bibliography analysis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001
7855076,1 Nano enabled drug delivery,Graphene,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000007
7855077,1 Nano enabled drug delivery,Specialization score,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7858840,fa(k)-index,fp(k)-index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001
7858841,fa(k)-index,fas(k)-index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001
7858842,fas(k)-index,fp(k)-index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000001
7858843,g-index,h-index,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000361


In [96]:
left = all_results['katz_beta001_t1'].copy()
right = df_betweenness.copy()

In [106]:
for k in  all_results.keys():
    all_results[k].to_pickle("output_scoresheet\\scoresheet_{}.pkl".format(k))

                link            no-link
                +--------------+---------------+
     predict    | tp           | fp            |
     value      | ret & rel    | ret & ~rel    |
                +--------------+---------------+
     no-predict | fn           | tn            |
     NA         | ~ret & rel   | ~ret & ~rel   |
                +--------------+---------------+

        Arguments
        ---------
        retrieved : a list or set
            iterable of the retrieved items

        relevant : a list or set
            iterable of the relevant items

        universe : a list or set, an int or None
            If universe is an iterable, it is interpreted as the set of all
            items in the system. If universe is an int, it is interpreted as
            the *number* of items in the system. This allows for fewer checks
            but is more memory-efficient. If universe is None, it is supposed
            to be unknown. This still allows for some measures, including
            precision and recall, to be calculated.


In [19]:
df_betweenness = df_betweenness.reset_index()
df_betweenness

,keyword1,keyword2,betweenness
0,All-in-one indicator,Altmetric score,0.000001
1,All-in-one indicator,Altmetrics,0.001344
2,1 Nano enabled drug delivery,Bibliography analysis,0.000001
3,1 Nano enabled drug delivery,Graphene,0.000007
4,1 Nano enabled drug delivery,Specialization score,0.000001
...,...,...,...
3767,fa(k)-index,fp(k)-index,0.000001
3768,fa(k)-index,fas(k)-index,0.000001
3769,fas(k)-index,fp(k)-index,0.000001
3770,g-index,h-index,0.000361
